In [0]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()  

In [0]:
!pip install split-folders
!rm -rf output

In [0]:
 !git clone https://github.com/pruvi007/ML_Datasets.git

fatal: destination path 'ML_Datasets' already exists and is not an empty directory.


In [0]:
import split_folders
split_folders.ratio('ML_Datasets/UCMerced_LandUse/Images', output="output", seed=1337, ratio=(.8, .2)) 

In [0]:
# unzip /content/101_ObjectCategories.zip

In [0]:
# import split_folders
# split_folders.ratio('101_ObjectCategories', output="output", seed=1337, ratio=(.6, .2, .2)) 

In [22]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(256),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(256),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'output'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
save_file_name = 'alexnet-transfer.pt'
len(class_names)

21

In [0]:
model = torchvision.models.alexnet(pretrained=True)

In [25]:
from torchsummary import summary
model.features
summary(model.features.cuda(), (3,256,256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 63, 63]          23,296
              ReLU-2           [-1, 64, 63, 63]               0
         MaxPool2d-3           [-1, 64, 31, 31]               0
            Conv2d-4          [-1, 192, 31, 31]         307,392
              ReLU-5          [-1, 192, 31, 31]               0
         MaxPool2d-6          [-1, 192, 15, 15]               0
            Conv2d-7          [-1, 384, 15, 15]         663,936
              ReLU-8          [-1, 384, 15, 15]               0
            Conv2d-9          [-1, 256, 15, 15]         884,992
             ReLU-10          [-1, 256, 15, 15]               0
           Conv2d-11          [-1, 256, 15, 15]         590,080
             ReLU-12          [-1, 256, 15, 15]               0
        MaxPool2d-13            [-1, 256, 7, 7]               0
Total params: 2,469,696
Trainable param

In [0]:
for param in model.parameters():
    param.requires_grad = False

In [0]:
model = model.features.to(device)

In [0]:
batch_size=32
train_size, validation_size = 1680, 420
def extract_features(phase, sample_count):
    features = np.zeros(shape=(sample_count, 256, 7, 7))  # Must be equal to the output of the convolutional base
    labels = np.zeros(shape=(sample_count))
   
    # Pass data through convolutional base
    i = 0
    for inputs, label in dataloaders[phase]:
        inputs = inputs.to(device)
        features_batch = model(inputs)
        features[i * batch_size: (i + 1) * batch_size] = features_batch.cpu().numpy()
        labels[i * batch_size: (i + 1) * batch_size] = label.numpy()
        i += 1
        if i * batch_size >= sample_count:
            break
    return features, labels
  
train_features, train_labels = extract_features('train', train_size) 
validation_features, validation_labels = extract_features('val', validation_size)

In [29]:
train_labels.shape,train_features.shape

((1680,), (1680, 256, 7, 7))

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC 
from scipy.ndimage import convolve
from sklearn import linear_model, datasets, metrics
from sklearn.model_selection import train_test_split
from sklearn.neural_network import BernoulliRBM
from sklearn.pipeline import Pipeline
from sklearn.base import clone

In [0]:
def scale(X, eps = 0.001):
	# scale the data points s.t the columns of the feature space
	# (i.e the predictors) are within the range [0, 1]
	return (X - np.min(X, axis = 0)) / (np.max(X, axis = 0) + eps)

In [32]:
X_train, y_train = train_features.reshape(1680,7*7*256), train_labels
X_train = scale(X_train)
X_test = scale(validation_features.reshape(420,7*7*256))
y_test = validation_labels
X_test.shape

(420, 12544)

In [0]:
# Models we will use
logistic = linear_model.LogisticRegression(solver='lbfgs', max_iter=10000,
                                           multi_class='multinomial')
rbm = BernoulliRBM(random_state=0, verbose=True)

rbm_features_classifier = Pipeline(
    steps=[('rbm', rbm), ('logistic', logistic)])

# #############################################################################
# Training

# Hyper-parameters. These were set by cross-validation,
# using a GridSearchCV. Here we are not performing cross-validation to
# save time.
rbm.learning_rate = 0.06
rbm.n_iter = 10
# More components tend to give better prediction performance, but larger
# fitting time
rbm.n_components = 500
logistic.C = 6000

In [34]:
# Training RBM-Logistic Pipeline
rbm_features_classifier.fit(X_train, y_train)

[BernoulliRBM] Iteration 1, pseudo-likelihood = -1819.74, time = 33.89s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -1740.85, time = 34.23s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -1226.41, time = 34.13s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -1185.92, time = 36.16s
[BernoulliRBM] Iteration 5, pseudo-likelihood = -1173.43, time = 34.19s
[BernoulliRBM] Iteration 6, pseudo-likelihood = -1143.90, time = 34.14s
[BernoulliRBM] Iteration 7, pseudo-likelihood = -1148.57, time = 34.13s
[BernoulliRBM] Iteration 8, pseudo-likelihood = -1152.20, time = 34.43s
[BernoulliRBM] Iteration 9, pseudo-likelihood = -1141.26, time = 34.68s
[BernoulliRBM] Iteration 10, pseudo-likelihood = -1147.74, time = 34.03s


Pipeline(memory=None,
     steps=[('rbm', BernoulliRBM(batch_size=10, learning_rate=0.06, n_components=500, n_iter=10,
       random_state=0, verbose=True)), ('logistic', LogisticRegression(C=6000, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=10000, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False))])

In [35]:
Y_pred = rbm_features_classifier.predict(X_test)
print("logistic using RBM features:\n%s\n" % (
    metrics.classification_report(y_test,Y_pred)))

logistic using RBM features:
              precision    recall  f1-score   support

         0.0       1.00      0.95      0.97        20
         1.0       0.75      0.60      0.67        20
         2.0       0.63      0.60      0.62        20
         3.0       0.95      1.00      0.98        20
         4.0       0.52      0.55      0.54        20
         5.0       1.00      1.00      1.00        20
         6.0       0.33      0.45      0.38        20
         7.0       1.00      0.85      0.92        20
         8.0       0.85      0.85      0.85        20
         9.0       0.61      0.55      0.58        20
        10.0       0.90      0.90      0.90        20
        11.0       0.52      0.65      0.58        20
        12.0       0.62      0.50      0.56        20
        13.0       0.76      0.65      0.70        20
        14.0       0.83      0.75      0.79        20
        15.0       0.95      1.00      0.98        20
        16.0       0.85      0.85      0.85        2

In [0]:
svm = SVC(C=1)
rbm = BernoulliRBM(random_state=0, verbose=True)
rbm_classifier = Pipeline(
    steps=[('rbm', rbm), ('svm', svm)])

rbm.learning_rate = 0.06
rbm.n_iter = 10
# More components tend to give better prediction performance, but larger
# fitting time
rbm.n_components = 500

In [37]:
# Training RBM-Logistic Pipeline
rbm_classifier.fit(X_train, y_train)

[BernoulliRBM] Iteration 1, pseudo-likelihood = -1819.74, time = 31.84s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -1740.85, time = 35.22s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -1226.41, time = 33.42s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -1185.92, time = 33.48s
[BernoulliRBM] Iteration 5, pseudo-likelihood = -1173.43, time = 33.35s
[BernoulliRBM] Iteration 6, pseudo-likelihood = -1143.90, time = 33.02s
[BernoulliRBM] Iteration 7, pseudo-likelihood = -1148.57, time = 32.98s
[BernoulliRBM] Iteration 8, pseudo-likelihood = -1152.20, time = 32.75s
[BernoulliRBM] Iteration 9, pseudo-likelihood = -1141.26, time = 33.06s
[BernoulliRBM] Iteration 10, pseudo-likelihood = -1147.74, time = 32.82s


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Pipeline(memory=None,
     steps=[('rbm', BernoulliRBM(batch_size=10, learning_rate=0.06, n_components=500, n_iter=10,
       random_state=0, verbose=True)), ('svm', SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False))])

In [38]:
Y_pred = rbm_classifier.predict(X_test)
print("SVM using RBM features:\n%s\n" % (
    metrics.classification_report(y_test,Y_pred)))

SVM using RBM features:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        20
         1.0       0.42      0.55      0.48        20
         2.0       0.56      0.45      0.50        20
         3.0       0.95      1.00      0.98        20
         4.0       0.57      0.60      0.59        20
         5.0       1.00      1.00      1.00        20
         6.0       0.33      0.05      0.09        20
         7.0       1.00      1.00      1.00        20
         8.0       0.69      0.45      0.55        20
         9.0       0.80      0.20      0.32        20
        10.0       0.95      0.90      0.92        20
        11.0       0.82      0.45      0.58        20
        12.0       0.48      0.75      0.59        20
        13.0       0.44      0.90      0.59        20
        14.0       0.58      0.90      0.71        20
        15.0       0.83      1.00      0.91        20
        16.0       0.74      0.70      0.72        20
   